# trigger correction

Given fast/continuously paced stimulus presentation and/or shortcomings of specific experimental software, the parallel port might occasionally send incorrect EEG event markers. This seems to depend on specific hardware-software interactions.  
Rather than catching and solving all such interactions, this posthoc fix aligns and matches behavioral log files (here in *.npz format) with the initial EEG event marker files in a semi-automatic way. It then produces an updated EEG event marker file.

## how to use

1. adjust the paths in the cell below as needed.
2. if you want to run all the files in the inpath directory, no further adjustments are needed  
alternatively, you can choose a subset of files based on:
    - sbjcode (default: "all"),
    - task (default: "all"), and 
    - group (default: "", i.e. no group).  

All these input arguments can be a single string value, or a list of strings.  
_note that groups parameter MUST be set explicitly if you have groups in the filename that determine file matching._  

3. run the cell below.
4. collect the files from the output folder and import to your EEG marker file with appropriate software. the naming of the files is the same as the filename stem of the EEG marker files.

## notes on plots (TODO: adjust documentation)

You can specify whether or not you want output diagnostic plots in this notebook. For that, set any of the parameters ``mismatch_plot`` in the call to ``BatchPosthocTriggerFix`` to **True** or **False** depending on your needs.  

### if mismatch_plot set to True (default; highly recommended)
With this diag plot, you may visually check the match of each file.  

![title](plot_1_illu.png)

- **middle black lines**: matches
- **green lines**: markers only in log file (i.e., *.npz)
- **red lines**: markers only in EEG marker file, but NOT ghost trigger
- **grey lines**: markers only in vmrk: "ghost" trigger (i.e. a trigger number not defined in decoding table)  

For example, it is a BAD indicator if N green lines >= N black lines (not counting green lines at beginning/end, esp. before 0).  
In that case, you should run another batch with tweaked parameters.  
Parameters to tweak are:
- **high_accuracy**: if True, slower calculations but possibly more accurate
- **only_middle**: default True; only consider the middle 50% of each for estimating matches. If False, consider all.
- **samp_uncertainty**: how many samples npz and vmrk are allowed to deviate to still be considered a match (in EEG samples (but TODO: double check))
- **nth_best**: the optimization process may in rare cases suggest a suboptimal solution (which should be visible in the mismatch_plot). if suspected that the first solution in this optimization process is in fact not the best, the user can change check the 2nd, 3rd, ... 513th, ... best
- **allow_manual_mode**: self-explanatory

# code cells

In [1]:
from trigfix.trigger_compare_functions import *

# either put EEG and logfiles into emp_fpath below, or adjust the path.
emp_fpath = root_path/"data"/"inpath"
outpath   = root_path/"data"/"outpath"

In [2]:
%matplotlib qt

In [3]:
%matplotlib widget

In [4]:
batch = BatchPosthocTriggerFix(
    lab="EEG2",
    inpath=emp_fpath, 
    outpath=outpath,
    samp_uncertainty=20,
    mismatch_plot=True,
    allow_manual_mode=False
    )

batch.apply_fix() #, sbjcodes="S14"

c:\users\nicade\documents\repos\trigger-fix\trigfix\globals.py:90: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  self.read_file()
